In [3]:
import os
import nltk

In [4]:
corpus_path = os.path.abspath("../data/NKJP_1.2_nltk/")
corpus = nltk.corpus.reader.TaggedCorpusReader(root=corpus_path, fileids=".*")

OSError: No such file or directory: '/Users/user/studia/zpp/data/NKJP_1.2_nltk'

In [5]:
tags = [x[1] for x in corpus.tagged_words()]
fqd = nltk.FreqDist(tags)



NameError: name 'corpus' is not defined

[('INTERP', 223499),
 ('QUB', 68402),
 ('CONJ', 44953),
 ('FIN:SG:TER:IMPERF', 30242),
 ('SUBST:SG:GEN:F', 29592),
 ('PREP:LOC:NWOK', 27118),
 ('SUBST:SG:NOM:M1', 23810),
 ('SUBST:SG:GEN:M3', 23296),
 ('SUBST:SG:NOM:F', 22481),
 ('ADV:POS', 21231),
 ('COMP', 17830),
 ('PREP:GEN', 17810),
 ('ADV', 17299),
 ('PREP:ACC', 17253),
 ('PREP:LOC', 15944),
 ('SUBST:SG:ACC:F', 15543),
 ('SUBST:SG:NOM:M3', 15161),
 ('SUBST:SG:LOC:M3', 14041),
 ('SUBST:SG:ACC:M3', 14034),
 ('PREP:GEN:NWOK', 13583),
 ('SUBST:SG:GEN:N', 13415),
 ('SUBST:SG:LOC:F', 12290),
 ('SUBST:SG:NOM:N', 11814),
 ('PRAET:SG:M1:PERF', 11078),
 ('SUBST:SG:ACC:N', 10673),
 ('INF:PERF', 10521),
 ('PREP:INST:NWOK', 10392),
 ('ADJ:SG:NOM:F:POS', 10094),
 ('SUBST:PL:GEN:M3', 9921),
 ('ADJ:SG:GEN:F:POS', 9804),
 ('FIN:PL:TER:IMPERF', 9599),
 ('SUBST:PL:GEN:F', 9478),
 ('ADJ:SG:GEN:M3:POS', 9444),
 ('SUBST:SG:GEN:M1', 9343),
 ('INF:IMPERF', 8698),
 ('BREV:PUN', 8277),
 ('PRAET:SG:M1:IMPERF', 7818),
 ('ADJ:SG:NOM:M3:POS', 7625),
 ('PREP:A

In [6]:
# @TODO merge tags so that their cardinalities are ≥  100, 1000
sets = fqd.most_common()
sets = [(s[0].split(':'), s[1]) for s in sets]
sets = [(s[0][0], s[0][1:], s[1]) for s in sets]
# sets = {s[0][0]:{'tags':s[0][1:], 'card': s[1]} for s in sets}
# we want a struct:
# main_tag : {
# list(
#     {tags: [other tags], card: cardinality}
# )
# }
structurized_data = {}
for s in sets:
    k = s[0]
    v = {'tags': s[1], 'card': s[2]}
    if k in structurized_data.keys():
        structurized_data[k]+=[v]
    else:
        structurized_data[k] =[v]
        
# structurized_data

We see that:
QUB has 1 optional tag, we'll want to remove it since they make a really small difference.  
PREP has 1 optional tag, but the sizes are ok, so we'll ignore it  
ADV is good as it is  
PRAET has 1 optional tag  
PPRON12 has 1 optional tag  

why am I mentioning this?
because the missing tags were breaking my naive algorithm lol.

In [22]:
# remove optional tags - in most cases their respective set sizes are too small anyway
keys_with_optional_tags = {'PREP', 'PRAET', 'PPRON12', 'QUB'}

def get_num_of_mandatory_tags(A):
    min_len = 0
    for el in A:
        if(len(el['tags'])<min_len):
            min_len = len(el['tags'])
    
    return min_len

for k in keys_with_optional_tags:
    flexem = structurized_data[k]
    mandatory_tags = get_num_of_mandatory_tags(flexem)
    for el in flexem:
        if len(el['tags'])>mandatory_tags:
            el['tags'] = el['tags'][:-1]
    structurized_data[k]=flexem

In [8]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

def are_equal(set1, set2):
    if len(set1)!=len(set2): return false
    res = True
    for e1, e2 in zip(set1, set2):
        res = res and (e1==e2)
    
    return res

In [39]:
from math import log
t = structurized_data['FIN'].copy()

def concat(A):
    res = ""
    for a in A:
        res += ':'
        res += a
    
    res = res[1:]
    return res


def min_size(A):
    min_s = 99999999;
    for el in A:
        if el['card'] < min_s:
            min_s = el['card']
            
    return min_s


def avg_size(A):
    res = 0
    for el in A:
        res += el['card']
        
    return res/len(A)


def cut_idx(A, idx):
    return [{'tags': e['tags'][:idx] + e['tags'][idx+1:], 'card': e['card']} for e in A]


def flatten(A):
    def merge(B):
        tmp = {'tags': B[0]['tags'], 'card': 0}
        for el in B:
            tmp['card'] += el['card']    
        return tmp
    
    sets = {}
    for el in A:
        key = concat(el['tags'])
        if key in sets:
            sets[key] += [el]
        else:
            sets[key] = [el]
    
    res = []
    for k, v in sets.items():
        res += [merge(v)]
    
    return res


def cut_one(t, best):
    n = len(t[0]['tags'])
    if n==0:
        return best
    for i in range(0, n):
        t = t.copy()
        cropped = cut_idx(t, i)
        cropped = flatten(cropped)
        
        min_s = min_size(cropped)
        
        if min_s > 40 and (min_s < best[0] or best[1]==-1):
            best = (min_s, i, cropped)
    
        # best = cut_one(cropped, best)
    
    return best


results = []
for k in structurized_data:
    t = structurized_data[k].copy()
    n = len(t[0]['tags'])
    
    t_flat = flatten(t)
            
    best = (min_size(t_flat), -1, t_flat)
    best = (9999999999999999, -1, None)
    best = cut_one(t, best)
        
    results += [(k, best[1], best[2])]
    
for r in results:
    print(r[0]+':')
    print(r[2])
    print(f'removed idx: {r[1]}')


INTERP:
None
removed idx: -1
QUB:
None
removed idx: -1
CONJ:
None
removed idx: -1
FIN:
[{'tags': ['SG', 'TER'], 'card': 34102}, {'tags': ['PL', 'TER'], 'card': 10818}, {'tags': ['SG', 'PRI'], 'card': 8241}, {'tags': ['PL', 'PRI'], 'card': 3171}, {'tags': ['SG', 'SEC'], 'card': 2808}, {'tags': ['PL', 'SEC'], 'card': 358}]
removed idx: 2
SUBST:
[{'tags': ['SG', 'GEN'], 'card': 76345}, {'tags': ['SG', 'NOM'], 'card': 74443}, {'tags': ['SG', 'ACC'], 'card': 44333}, {'tags': ['SG', 'LOC'], 'card': 32862}, {'tags': ['PL', 'GEN'], 'card': 32755}, {'tags': ['SG', 'INST'], 'card': 19730}, {'tags': ['PL', 'NOM'], 'card': 17500}, {'tags': ['PL', 'ACC'], 'card': 15147}, {'tags': ['PL', 'LOC'], 'card': 7136}, {'tags': ['PL', 'INST'], 'card': 4603}, {'tags': ['SG', 'DAT'], 'card': 3699}, {'tags': ['SG', 'VOC'], 'card': 1421}, {'tags': ['PL', 'DAT'], 'card': 1646}]
removed idx: 2
PREP:
None
removed idx: -1
ADV:
[{'tags': [], 'card': 42340}]
removed idx: 0
COMP:
None
removed idx: -1
PRAET:
None
remove

In [11]:
adj = structurized_data['ADJ'].copy()
# adj
for i in range(0, 3):
    ad = adj.copy()
    ad = cut_idx(ad, i)
    ad = flatten(ad)
    print(ad)
    print('--------------------')
    for j in range(0, 2):
        ad2 = ad.copy()
        ad2 = cut_idx(ad2, j)
        ad2 = flatten(ad2)
        print(ad2)
        print('-----------------------------')


[{'tags': ['NOM', 'F', 'POS'], 'card': 13202}, {'tags': ['GEN', 'F', 'POS'], 'card': 13984}, {'tags': ['GEN', 'M3', 'POS'], 'card': 13078}, {'tags': ['NOM', 'M3', 'POS'], 'card': 10231}, {'tags': ['NOM', 'N', 'POS'], 'card': 7519}, {'tags': ['LOC', 'M3', 'POS'], 'card': 7053}, {'tags': ['ACC', 'M3', 'POS'], 'card': 8111}, {'tags': ['ACC', 'F', 'POS'], 'card': 8030}, {'tags': ['NOM', 'M1', 'POS'], 'card': 8448}, {'tags': ['LOC', 'F', 'POS'], 'card': 5706}, {'tags': ['GEN', 'N', 'POS'], 'card': 6198}, {'tags': ['ACC', 'N', 'POS'], 'card': 4379}, {'tags': ['INST', 'F', 'POS'], 'card': 3066}, {'tags': ['INST', 'M3', 'POS'], 'card': 2905}, {'tags': ['LOC', 'N', 'POS'], 'card': 2585}, {'tags': ['GEN', 'M1', 'POS'], 'card': 3007}, {'tags': ['INST', 'N', 'POS'], 'card': 1502}, {'tags': ['INST', 'M1', 'POS'], 'card': 1060}, {'tags': ['ACC', 'M1', 'POS'], 'card': 1321}, {'tags': ['DAT', 'F', 'POS'], 'card': 623}, {'tags': ['NOM', 'M2', 'POS'], 'card': 628}, {'tags': ['DAT', 'M1', 'POS'], 'card':